#### Reading the input excel files.

In [ ]:
import pandas as pd
import re
import sqlite3
import networkx as nx
from openpyxl import load_workbook

uploaded_file = r"C:\Users\ADMIN\Desktop\AI_XL_Agent\data\valuation_test_case.xlsx"
uploaded_file_1 = r"C:\Users\ADMIN\Desktop\AI_XL_Agent\data\multi_header_test.xlsx"
sheets = pd.read_excel(uploaded_file_1, sheet_name=None) 




#### Importing helper functions to re format the data before saving to db

In [ ]:
from excel_ingestion_process import reshape_wide, is_wide_format

In [ ]:
normalized_sheets = {}
for sheet, df in sheets.items():
    normalized_sheets[sheet] = reshape_wide(df) if is_wide_format(df) else df
    
conn = sqlite3.connect("workbook.db")

for sheet, df in normalized_sheets.items():
    df.to_sql(sheet, conn, if_exists="replace", index=False)

In [ ]:
import openpyxl


# Load the workbook (data_only=False ensures we see formulas, not just values)
wb = openpyxl.load_workbook(uploaded_file, data_only=False)



#### This is from a github example, where a dependancy graph is build to understand the formulae and relations in a excel workbook-- we re write this later for a better result.

In [7]:
import openpyxl
import re
import networkx as nx
import matplotlib.pyplot as plt
from typing import List, Dict, Set

class ExcelDependencyGraph:
    def __init__(self, file_path: str):
        self.file_path = file_path
        self.wb = openpyxl.load_workbook(file_path, data_only=False)
        self.graph = nx.DiGraph()
        # Pattern matches single cells (e.g., A1), ranges (e.g., A1:B2), and supports absolute refs
        self.formula_pattern = r'(\b[A-Z]+\$?\d+\b)'

    def parse_worksheet(self, sheet_name: str) -> None:
        """Parse a worksheet and add cells and dependencies to the graph."""
        sheet = self.wb[sheet_name]

        for row in sheet.iter_rows():
            for cell in row:
                if cell.value and isinstance(cell.value, str) and cell.value.startswith('='):
                    cell_coord = f"{sheet_name}!{cell.coordinate}"
                    formula = cell.value
                    dependencies = self._extract_dependencies(formula, sheet_name)
                    
                    # Add each dependency as an edge in the graph
                    for dep in dependencies:
                        self.graph.add_edge(dep, cell_coord)

    def _extract_dependencies(self, formula: str, sheet_name: str) -> Set[str]:
        """Extract cell dependencies from a formula using regex and return them as full sheet-cell refs."""
        matches = re.findall(self.formula_pattern, formula)
        dependencies = set()
        for match in matches:
            # Ensure each dependency is prefixed with the sheet name
            if '!' not in match:
                match = f"{sheet_name}!{match}"
            dependencies.add(match)
        return dependencies

    def build_dependency_graph(self) -> None:
        """Build the dependency graph by parsing each sheet in the workbook."""
        for sheet_name in self.wb.sheetnames:
            self.parse_worksheet(sheet_name)

    def visualize_graph(self) -> None:
        """Visualize the dependency graph."""
        plt.figure(figsize=(100,100))
        pos = nx.shell_layout(self.graph)
        nx.draw_networkx(self.graph, pos, with_labels=True, node_size = 5000, node_color="skyblue", font_size=10, arrowsize = 15,  arrows=True)
        plt.title("Excel Cell Dependency Graph")
        plt.show()

    def get_dependencies(self, cell: str) -> List[str]:
        """Return a list of cells that the given cell depends on."""
        if cell in self.graph:
            return list(self.graph.predecessors(cell))
        return []

    def get_dependents(self, cell: str) -> List[str]:
        """Return a list of cells that depend on the given cell."""
        if cell in self.graph:
            return list(self.graph.successors(cell))
        return []

    def save_graph(self, output_path: str) -> None:
        """Save the dependency graph to a file in GML format."""
        nx.write_gml(self.graph, output_path)



In [14]:
graph = ExcelDependencyGraph("data/valuation_test_case.xlsx")
graph.build_dependency_graph()
graph.visualize_graph()
print(graph.get_dependencies("IncomeStmt!G2"))
print(graph.get_dependents("Sheet1!A1"))
graph.save_graph("dependency_graph.gml")

['IncomeStmt!F2']
[]


Getting all the prompts from the prompts file

In [13]:
from prompts import XL_Agent_prompt, CLASSIFIER_AGENT_SYSTEM_PROMPT, TABLE_SELECTION_SYSTEM_PROMPT, SQL_GENERATION_SYSTEM_PROMPT, GRAPH_RELEVANT_SHEETS_IDENTIFIER_PROMPT,GRAPH_EXECUTION_SYSTEM_PROMPT, CODE_GENERATION_PROMPT

Activating gemini api key

In [8]:
import google.generativeai as genai
import copy
from config import GEMINI_API_KEY, excel_path, db_path
import ast
genai.configure(api_key= GEMINI_API_KEY)

model = genai.GenerativeModel("gemini-2.5-flash")


User questions

In [112]:
user_question = "What is EBITDA in 2021 and 2022?"
user_question_1 = "How is EBIT calculated in 2026?"
user_question_2 = "Which sheets reference IncomeStmt!B2?"

In [10]:
q1 = "Sum FY2024 revenue by region across sheets"

To classify the question into sql or graph based

In [ ]:
CLASSIFIER_AGENT_SYSTEM_PROMPT_COPY = copy.deepcopy(CLASSIFIER_AGENT_SYSTEM_PROMPT)

formatted_prompt_text=CLASSIFIER_AGENT_SYSTEM_PROMPT_COPY.format(
    user_question= user_question
)
# print(formatted_prompt_text)

In [50]:
response = model.generate_content(formatted_prompt_text)

In [ ]:
classified_response=response.text
print(classified_response)


Trying to get all the table schemas and also the 1st few rows as a sample, to help in deciding the correct table from which the query should be generated.

In [3]:
import sqlite3
from typing import Dict, Tuple, List

def get_all_table_schemas_with_samples(
    db_file: str,
    sample_rows: int = 2,
    tables: List[str] = None
) -> Tuple[Dict[str, Dict[str, str]], str]:
    """
    Extracts schema and sample rows from specified tables in a SQLite database.

    Args:
        db_file: Path to the SQLite database file.
        sample_rows: Number of sample rows to retrieve per table (default = 2).
        tables: Optional list of table names. If empty or None, all tables are used.

    Returns:
        - dict mapping table_name -> { "schema": str, "sample": str }
        - formatted string suitable for LLM prompts including schema + sample rows
    """
    table_info = {}
    formatted_schemas = []

    with sqlite3.connect(db_file) as conn:
        cursor = conn.cursor()

        # If no specific tables are provided, fetch all table names
        if not tables:
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
            tables = [row[0] for row in cursor.fetchall()]

        for table in tables:
            # Check if the table actually exists
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table,))
            if not cursor.fetchone():
                print(f"⚠️ Warning: Table '{table}' does not exist in the database.")
                continue

            # Get schema
            cursor.execute(f"PRAGMA table_info({table})")
            columns = cursor.fetchall()
            if not columns:
                continue

            col_strings = [f"{col[1]} ({col[2]})" for col in columns]
            schema_str = ", ".join(col_strings)

            # Get sample rows
            cursor.execute(f'SELECT * FROM "{table}" LIMIT {sample_rows}')
            rows = cursor.fetchall()
            col_names = [desc[0] for desc in cursor.description]

            if rows:
                sample_str = " | ".join(col_names) + "\n"
                sample_str += "\n".join(" | ".join(str(v) for v in row) for row in rows)
            else:
                sample_str = "No sample rows available"

            # Store info
            table_info[table] = {"schema": schema_str, "sample": sample_str}

            # Build formatted output
            formatted_schemas.append(
                f"Schema for table '{table}': {schema_str}\nSample rows:\n{sample_str}"
            )

    formatted_schemas_str = "\n\n".join(formatted_schemas)
    return table_info, formatted_schemas_str


In [1]:
from config import excel_path, db_path

In [118]:
table_info, formatted_schemas_str = get_all_table_schemas_with_samples(db_path)

In [119]:
response_table_selection=None

If the question is a 'SQL'related question, we get the related tables and schemas

In [ ]:
if classified_response == "SQL":
    TABLE_SELECTION_SYSTEM_PROMPT_COPY = copy.deepcopy(TABLE_SELECTION_SYSTEM_PROMPT)
    
    
    
    formatted_prompt_text_table_selection=TABLE_SELECTION_SYSTEM_PROMPT_COPY.format(
        user_question=user_question,
        tables_and_schema=formatted_schemas_str
    )
    print(formatted_prompt_text_table_selection)
    response_table_selection = model.generate_content(formatted_prompt_text_table_selection)



You are a SQL table selection assistant.  

You are given:
1. A user question.
2. Multiple tables with their schema and sample data.

Your job:
- Identify which table(s) contain the data relevant to the question.
- Output only the table name(s) in a JSON list.
- Donot hallucinate any table names.

Do not generate SQL yet.

Example:

Question: "What is EBITDA in 2021 and 2022?"

Tables:
IncomeStmt
Columns: LineItem(TEXT), Year(TEXT), Value(REAL)
Sample data:
  LineItem  Year   Value
0 Revenue   2021   800
1 EBITDA    2021   192
2 D&A       2021   32

CashFlow
Columns: LineItem(TEXT), Year(TEXT), Value(REAL)
Sample data:
  LineItem  Year   Value
0 NOPAT     2021   120
1 CapEx     2021   40

Output: ["IncomeStmt"]

---

Input:  
Question: What is EBITDA in 2021 and 2022? 
Tables: Schema for table 'Assumptions': Name (TEXT), Value (REAL), Notes (TEXT)
Sample rows:
Name | Value | Notes
WACC | 0.1 | Weighted Avg. Cost of Capital
TerminalGrowth | 0.03 | Long-term perpetual growth

Schema fo

In [124]:
relevant_tables= ast.literal_eval(response_table_selection.text)

In [125]:
_,relevant_tables_schema_str = get_all_table_schemas_with_samples(db_path, tables=relevant_tables)

In [24]:
q1

'Sum FY2024 revenue by region across sheets'

In [126]:
print(relevant_tables_schema_str)

Schema for table 'IncomeStmt': Line Item (TEXT), Time (TEXT), Value (REAL)
Sample rows:
Line Item | Time | Value
Revenue | 2021 | 800.0
EBITDA | 2021 | 192.0


#### The following is to generate a sql query based on the user query and the relevant tables generated.

In [127]:
SQL_GENERATION_SYSTEM_PROMPT_COPY = copy.deepcopy(SQL_GENERATION_SYSTEM_PROMPT)    
    
formatted_prompt_text_sql_query_generation=SQL_GENERATION_SYSTEM_PROMPT_COPY.format(
    user_question=user_question,
    relevant_tables_and_schema=relevant_tables_schema_str
)
print(formatted_prompt_text_sql_query_generation)
response_sql_query = model.generate_content(formatted_prompt_text_sql_query_generation)


You are a SQL query generator.  

You are given:
1. User question
2. The relevant table(s) schema and top rows

Generate a valid SQL query to answer the question.
The query should be valid and should not contain any syntax errors.
Even if the question is not directly answerable from the top rows, generate a valid SQL query.
Do NOT include explanations, markdown, or code block formatting.
Output the raw SQL query text only — no ```sql, no quotes, no extra text.

Example:

Question: "What is EBITDA in 2021 and 2022?"

Table: IncomeStmt
Columns: LineItem(TEXT), Year(TEXT), Value(REAL)
Sample data:
  LineItem  Year   Value
0 Revenue   2021   800
1 EBITDA    2021   192
2 D&A       2021   32

Output: 
SELECT Year, Value
FROM IncomeStmt
WHERE LineItem = 'EBITDA'
AND Year IN ('2021', '2022');

---

Input:  
Question: What is EBITDA in 2021 and 2022? 
Tables: Schema for table 'IncomeStmt': Line Item (TEXT), Time (TEXT), Value (REAL)
Sample rows:
Line Item | Time | Value
Revenue | 2021 | 800.0


In [128]:
sql_query=response_sql_query.text

This function is to execute the sql query.

In [33]:
import sqlite3
import pandas as pd
from typing import Union, List

def execute_sql_query(db_file: str, sql_query: str) -> Union[pd.DataFrame, str]:
    """
    Executes a SQL query against a SQLite database and returns the result as a DataFrame.

    Args:
        db_file (str): Path to the SQLite database file.
        sql_query (str): The SQL query string to execute.

    Returns:
        - pandas.DataFrame: If the query returns rows (e.g., SELECT).
        - str: A success message (e.g., for UPDATE, INSERT, DELETE, CREATE).
    """
    try:
        # Connect to the database
        with sqlite3.connect(db_file) as conn:
            cursor = conn.cursor()

            # Check if it's a SELECT query
            if sql_query.strip().lower().startswith("select"):
                df = pd.read_sql_query(sql_query, conn)
                return df
            else:
                cursor.execute(sql_query)
                conn.commit()
                return "✅ Query executed successfully (no result set)."

    except sqlite3.Error as e:
        return f"❌ SQLite error: {e}"
    except Exception as e:
        return f"❌ Unexpected error: {e}"


In [35]:
sql_query

'SELECT Region, SUM("FY2024 Revenue")\nFROM Data\nGROUP BY Region;'

In [ ]:
result = execute_sql_query(db_path, sql_query)
if isinstance(result, str):
    print(result)  # Error or success message
else:
    print(result)  # DataFrame

   Time  Value
0  2021  192.0
1  2022  216.0


The following is to generate a human understandable answer from the query and results formed.

In [131]:
FINAL_ANSWER_GENERATION_PROMPT_COPY = copy.deepcopy(FINAL_ANSWER_GENERATION_PROMPT)    
    
final_answer_generation_prompt=FINAL_ANSWER_GENERATION_PROMPT_COPY.format(
    user_question=user_question,
    sql_query= sql_query,
    sql_query_result = result
)
# print(formatted_prompt_text_sql_query_generation)
final_answer = model.generate_content(final_answer_generation_prompt)

In [132]:
final_answer.text

'EBITDA was 192.0 in 2021 and increased to 216.0 in 2022. This represents a growth of 24.0, or approximately 12.5%, from 2021 to 2022.'

#### Builds a directed graph of Excel cell dependencies with context.
#### Each node is a cell, enriched with row label, column header (e.g., year), value, and formula.
#### Edges represent formula-based dependencies between cells.

In [52]:
import openpyxl
import networkx as nx
import re

def build_contextual_dependency_graph(excel_path: str):
    wb = openpyxl.load_workbook(excel_path, data_only=False)
    G = nx.DiGraph()

    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]

        # Get header row (first row) and row labels (first column)
        headers = [cell.value for cell in sheet[1]]  # e.g. ['Line Item', 2021, 2022, 2023, 2024]

        for row in sheet.iter_rows(min_row=2):  # skip header row
            row_label = row[0].value  # first column, e.g. "EBITDA"

            for col_idx, cell in enumerate(row[1:], start=2):  # skip "Line Item" column
                coord = f"{sheet_name}!{cell.coordinate}"
                header = headers[col_idx-1] if col_idx-1 < len(headers) else None

                G.add_node(coord, 
                           sheet=sheet_name,
                           label=row_label,
                           year=header,
                           value=cell.value,
                           formula=cell.value if isinstance(cell.value, str) and cell.value.startswith("=") else None)

                # Parse dependencies if formula exists
                if isinstance(cell.value, str) and cell.value.startswith('='):
                    refs = re.findall(r"([A-Za-z0-9_]+![A-Z]+\d+|[A-Z]+\d+)", cell.value)
                    for ref in refs:
                        if '!' not in ref:
                            ref = f"{sheet_name}!{ref}"
                        G.add_edge(ref, coord, formula=cell.value)

    return G


In [53]:
graph = build_contextual_dependency_graph(excel_path)
graph

In [54]:
for u, v, data in list(graph.edges(data=True))[:10]:
    print(f"{u} -> {v}, formula: {data.get('formula')}")


IncomeStmt!B2 -> IncomeStmt!B3, formula: =B2*EBITDA_Margin
IncomeStmt!B2 -> IncomeStmt!B4, formula: =B2*DA_Percent
IncomeStmt!B2 -> CashFlow!C4, formula: =(IncomeStmt!C2*NWC_Percent)-(IncomeStmt!B2*NWC_Percent)
IncomeStmt!B2 -> CashFlow!B5, formula: =IncomeStmt!B2*Capex_Percent
IncomeStmt!C2 -> IncomeStmt!C3, formula: =C2*EBITDA_Margin
IncomeStmt!C2 -> IncomeStmt!C4, formula: =C2*DA_Percent
IncomeStmt!C2 -> CashFlow!C4, formula: =(IncomeStmt!C2*NWC_Percent)-(IncomeStmt!B2*NWC_Percent)
IncomeStmt!C2 -> CashFlow!D4, formula: =(IncomeStmt!D2*NWC_Percent)-(IncomeStmt!C2*NWC_Percent)
IncomeStmt!C2 -> CashFlow!C5, formula: =IncomeStmt!C2*Capex_Percent
IncomeStmt!D2 -> IncomeStmt!D3, formula: =D2*EBITDA_Margin


In [33]:
graph.nodes

NodeView(('Assumptions!B2', 'Assumptions!C2', 'Assumptions!B3', 'Assumptions!C3', 'Assumptions!B4', 'Assumptions!C4', 'Assumptions!B5', 'Assumptions!C5', 'Assumptions!B6', 'Assumptions!C6', 'Assumptions!B7', 'Assumptions!C7', 'Assumptions!B8', 'Assumptions!C8', 'Assumptions!B9', 'Assumptions!C9', 'Assumptions!B10', 'Assumptions!C10', 'IncomeStmt!B2', 'IncomeStmt!C2', 'IncomeStmt!D2', 'IncomeStmt!E2', 'IncomeStmt!F2', 'IncomeStmt!G2', 'IncomeStmt!H2', 'IncomeStmt!B3', 'IncomeStmt!C3', 'IncomeStmt!D3', 'IncomeStmt!E3', 'IncomeStmt!F3', 'IncomeStmt!G3', 'IncomeStmt!H3', 'IncomeStmt!B4', 'IncomeStmt!C4', 'IncomeStmt!D4', 'IncomeStmt!E4', 'IncomeStmt!F4', 'IncomeStmt!G4', 'IncomeStmt!H4', 'IncomeStmt!B5', 'IncomeStmt!C5', 'IncomeStmt!D5', 'IncomeStmt!E5', 'IncomeStmt!F5', 'IncomeStmt!G5', 'IncomeStmt!H5', 'CashFlow!B2', 'CashFlow!C2', 'CashFlow!D2', 'CashFlow!E2', 'CashFlow!F2', 'CashFlow!G2', 'CashFlow!H2', 'CashFlow!B3', 'CashFlow!C3', 'CashFlow!D3', 'CashFlow!E3', 'CashFlow!F3', 'CashFlo

In [56]:
import networkx as nx

def filter_graph_by_relevance(G, relevant_sheets=None, relevant_nodes=None):
    """
    Create a filtered subgraph based on relevant sheet names or node IDs.
    Keeps all edges connected to relevant nodes.
    """
    relevant_sheets = set(relevant_sheets or [])
    relevant_nodes = set(relevant_nodes or [])

    # Filter nodes by sheet or by explicit node names
    selected_nodes = [
        n for n, md in G.nodes(data=True)
        if md.get("sheet") in relevant_sheets or n in relevant_nodes
    ]

    # Also include their neighbors for full dependency context
    neighbors = set(selected_nodes)
    for node in selected_nodes:
        neighbors.update(G.predecessors(node))
        neighbors.update(G.successors(node))

    subgraph = G.subgraph(neighbors).copy()
    return subgraph


In [57]:
GRAPH_RELEVANT_SHEETS_IDENTIFIER_PROMPT = """
You are an intelligent assistant for an Excel comprehension system.

You are given a user's natural language question about an Excel workbook.
Your job is to identify which specific sheet(s), table(s), or cell(s) the question refers to.

Example: 

If the question explicitly mentions a cell (like "IncomeStmt!B2"), extract the sheet and cell reference.
If it only mentions a sheet or concept (like "How is EBIT calculated?"), identify the relevant sheet(s) or label(s) that most likely contain that concept.

Return the result strictly in JSON format as follows:
{{
  "relevant_sheets": ["Sheet1", "Sheet2"],
  "relevant_cells": ["Sheet1!A1", "Sheet2!B5"],
  "keywords": ["EBIT", "Revenue"]
}}

---
Input:
Question: {user_question}

Output: JSON object with relevant sheets, cells, and keywords (DONOT give any extra text or explanation or markdown formatting).
"""


In [59]:
user_question_2

'Which sheets reference IncomeStmt!B2?'

#### To identify which sheets the user question refers to, so that we can filter them out.

In [58]:
GRAPH_RELEVANT_SHEETS_IDENTIFIER_PROMPT_COPY = copy.deepcopy(GRAPH_RELEVANT_SHEETS_IDENTIFIER_PROMPT)    
    
formatted_prompt_graph_relevant_sheets_identifier=GRAPH_RELEVANT_SHEETS_IDENTIFIER_PROMPT_COPY.format(
    user_question=user_question_2
)
print(formatted_prompt_graph_relevant_sheets_identifier)
response_relevant_sheets = model.generate_content(formatted_prompt_graph_relevant_sheets_identifier)


You are an intelligent assistant for an Excel comprehension system.

You are given a user's natural language question about an Excel workbook.
Your job is to identify which specific sheet(s), table(s), or cell(s) the question refers to.

Example: 

If the question explicitly mentions a cell (like "IncomeStmt!B2"), extract the sheet and cell reference.
If it only mentions a sheet or concept (like "How is EBIT calculated?"), identify the relevant sheet(s) or label(s) that most likely contain that concept.

Return the result strictly in JSON format as follows:
{
  "relevant_sheets": ["Sheet1", "Sheet2"],
  "relevant_cells": ["Sheet1!A1", "Sheet2!B5"],
  "keywords": ["EBIT", "Revenue"]
}

---
Input:
Question: Which sheets reference IncomeStmt!B2?

Output: JSON object with relevant sheets, cells, and keywords (DONOT give any extra text or explanation or markdown formatting).



In [61]:
relevant_nodes_sheets = ast.literal_eval(response_relevant_sheets.text)
relevant_nodes_sheets 

{'relevant_sheets': ['IncomeStmt'],
 'relevant_cells': ['IncomeStmt!B2'],
 'keywords': []}

In [62]:
relevant_sheets = relevant_nodes_sheets["relevant_sheets"]

#### From the above relevant sheets, this helps in filtering out the relevant graph part and returns a sub graph which narrows down our search part

In [ ]:
import networkx as nx

def filter_graph_by_relevance(G, relevant_sheets=None, relevant_nodes=None):
    """
    Create a filtered subgraph based on relevant sheet names or node IDs.
    Keeps all edges connected to relevant nodes.
    """
    relevant_sheets = set(relevant_sheets or [])
    relevant_nodes = set(relevant_nodes or [])

    # Filter nodes by sheet or by explicit node names
    selected_nodes = [
        n for n, md in G.nodes(data=True)
        if md.get("sheet") in relevant_sheets or n in relevant_nodes
    ]

    # Also include their neighbors for full dependency context
    neighbors = set(selected_nodes)
    for node in selected_nodes:
        neighbors.update(G.predecessors(node))
        neighbors.update(G.successors(node))

    subgraph = G.subgraph(neighbors).copy()
    return subgraph

#### From the above sub-graph, this returns a human-readable textual representation of the sub graph, showing nodes with their important attributes and edges with formulas.

In [ ]:
def serialize_graph_text(G):
    """
    Returns a human-readable textual representation of the entire graph,
    showing nodes with their important attributes and edges with formulas.
    """
    lines = ["=== Nodes ==="]
    for n, md in G.nodes(data=True):
        lines.append(
            f"- {n} | label={md.get('label')} | header={md.get('header')} | value={md.get('value')} | formula={md.get('formula')}"
        )

    lines.append("\n=== EDGES ===")
    for u, v, data in G.edges(data=True):
        lines.append(f"{u} -> {v} | via formula: {data.get('formula')}")

    return "\n".join(lines)

In [63]:
subgraph = filter_graph_by_relevance(graph, relevant_nodes_sheets["relevant_sheets"], relevant_nodes_sheets["relevant_cells"])
subgraph_text = serialize_graph_text(subgraph)

In [64]:
print(subgraph_text)

=== Nodes ===
- CashFlow!H4 | label=Change in NWC | header=None | value==(IncomeStmt!H2*NWC_Percent)-(IncomeStmt!G2*NWC_Percent) | formula==(IncomeStmt!H2*NWC_Percent)-(IncomeStmt!G2*NWC_Percent)
- IncomeStmt!B4 | label=D&A | header=None | value==B2*DA_Percent | formula==B2*DA_Percent
- CashFlow!F4 | label=Change in NWC | header=None | value==(IncomeStmt!F2*NWC_Percent)-(IncomeStmt!E2*NWC_Percent) | formula==(IncomeStmt!F2*NWC_Percent)-(IncomeStmt!E2*NWC_Percent)
- IncomeStmt!G2 | label=Revenue | header=None | value==F2*(1+TerminalGrowth) | formula==F2*(1+TerminalGrowth)
- IncomeStmt!D2 | label=Revenue | header=None | value=1000 | formula=None
- CashFlow!G3 | label=D&A | header=None | value==IncomeStmt!G4 | formula==IncomeStmt!G4
- IncomeStmt!F3 | label=EBITDA | header=None | value==F2*EBITDA_Margin | formula==F2*EBITDA_Margin
- IncomeStmt!G3 | label=EBITDA | header=None | value==G2*EBITDA_Margin | formula==G2*EBITDA_Margin
- IncomeStmt!D3 | label=EBITDA | header=None | value==D2*EBITD

#### The following results in the final answer from the user query and the filtered sub graph

In [65]:
GRAPH_EXECUTION_SYSTEM_PROMPT_COPY = copy.deepcopy(GRAPH_EXECUTION_SYSTEM_PROMPT)    
    
formatted_prompt_graph_execution_system=GRAPH_EXECUTION_SYSTEM_PROMPT_COPY.format(
    user_question=user_question_2,
    filtered_dependency_graph=subgraph_text
)
print(formatted_prompt_graph_execution_system)
graph_sytem_output = model.generate_content(formatted_prompt_graph_execution_system)


You are a formula reasoning assistant.  
You are given:  
1. A filtered dependency graph of formulas and values from an Excel workbook.  
2. A user’s question.  

Your task:  
- Traverse the graph to determine how the requested value is computed.  
- Provide a step-by-step explanation of the formula.  
- If possible, compute the numeric result as well.  

----

Input:
Graph: === Nodes ===
- CashFlow!H4 | label=Change in NWC | header=None | value==(IncomeStmt!H2*NWC_Percent)-(IncomeStmt!G2*NWC_Percent) | formula==(IncomeStmt!H2*NWC_Percent)-(IncomeStmt!G2*NWC_Percent)
- IncomeStmt!B4 | label=D&A | header=None | value==B2*DA_Percent | formula==B2*DA_Percent
- CashFlow!F4 | label=Change in NWC | header=None | value==(IncomeStmt!F2*NWC_Percent)-(IncomeStmt!E2*NWC_Percent) | formula==(IncomeStmt!F2*NWC_Percent)-(IncomeStmt!E2*NWC_Percent)
- IncomeStmt!G2 | label=Revenue | header=None | value==F2*(1+TerminalGrowth) | formula==F2*(1+TerminalGrowth)
- IncomeStmt!D2 | label=Revenue | header=No

In [66]:
print(graph_sytem_output.text)

The formula `IncomeStmt!B2` is referenced by formulas in the following sheets:

1.  **IncomeStmt**:
    *   `IncomeStmt!B3` uses `IncomeStmt!B2` in its formula `=B2*EBITDA_Margin`.
    *   `IncomeStmt!B4` uses `IncomeStmt!B2` in its formula `=B2*DA_Percent`.
2.  **CashFlow**:
    *   `CashFlow!C4` uses `IncomeStmt!B2` in its formula `=(IncomeStmt!C2*NWC_Percent)-(IncomeStmt!B2*NWC_Percent)`.
    *   `CashFlow!B5` uses `IncomeStmt!B2` in its formula `=IncomeStmt!B2*Capex_Percent`.

Therefore, the sheets that reference `IncomeStmt!B2` are **IncomeStmt** and **CashFlow**.


In [71]:
excel_path = r"C:\Users\KUNDURI\Downloads\AI_XL_Agent\data\valuation_test_case_final.xlsx"


#### The following prompt is to generate a python code that will take in a excel file, query to make changes to the file accordingly and regenerate a new file.

In [108]:
CODE_GENERATION_PROMPT = """
You are an intelligent Excel code generation assistant.
Your task is to generate code that transforms Excel files exactly as requested by the user.

Guidelines:

- The user will describe a transformation or modification they want applied to an Excel file.
- You must generate only the raw executable code that performs the transformation — no explanations, no markdown, no comments, and no additional text.
- The output code should read an Excel file, apply the transformation, and save the result as a new Excel file.
- If the workbook contains multiple sheets, identify and operate on the specific sheet(s) mentioned by the user.
- Maintain the integrity of data in all other sheets unless explicitly instructed otherwise.
- When adding new columns (e.g., future quarters), derive values using linear extrapolation or trend analysis based on previous columns unless the user specifies exact values.
- When the user requests a change to a parameter (e.g., Capex_Percent), ensure the updated value is used in all calculations and overrides any previous default or assumed values.
- Do not retain or reuse outdated values unless explicitly instructed.
- Do not default to zero unless the user explicitly requests it.
- Preserve formulas, formatting, and schema integrity across sheets.
- Add new tables while preserving schema and formulas.
- Do not hallucinate or assume unrelated data; rebuild linked sheets only if instructed.

Output Rule:
Output must contain only the code text, with no markdown syntax, no quotes, and no explanations.

---

Input:  
XL file: {xl_file}  
Question: {user_question}  

Output: (code only, no markdown or explanation).



"""

In [109]:
CODE_GENERATION_PROMPT_COPY = copy.deepcopy(CODE_GENERATION_PROMPT)    
    
formatted_code_generation_prompt=   CODE_GENERATION_PROMPT_COPY.format(
    xl_file= excel_path,
    user_question="Increase Capex_Percent to 6% and regenerate valuation"
)
code_generation_output = model.generate_content(formatted_code_generation_prompt)

In [110]:
print(code_generation_output.text)

import openpyxl
import os
input_filepath = r"C:\Users\KUNDURI\Downloads\AI_XL_Agent\data\valuation_test_case_final.xlsx"
output_filepath = r"C:\Users\KUNDURI\Downloads\AI_XL_Agent\data\valuation_regenerated_capex_6pct.xlsx"
new_capex_value = 0.06
wb = openpyxl.load_workbook(input_filepath)
capex_percent_found = False
if 'Capex_Percent' in wb.defined_names:
    named_range = wb.defined_names['Capex_Percent']
    for sheet_name, cell_address in named_range.destinations:
        ws = wb[sheet_name]
        cell = ws[cell_address]
        cell.value = new_capex_value
        capex_percent_found = True
        break
if not capex_percent_found:
    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]
        for row in ws.iter_rows():
            for cell in row:
                if isinstance(cell.value, str) and "capex_percent" in cell.value.lower():
                    if cell.column < ws.max_column:
                        target_cell = ws.cell(row=cell.row, column=cell.column + 1

#### The following is the final step that will save the cleaned code in a .py file and execute it to get the final code.

In [111]:
generated_code = code_generation_output.text.strip()

# 2️⃣ Save the generated code for logging/debugging
with open("generated_transform.py", "w", encoding="utf-8") as f:
    f.write(generated_code)

# 3️⃣ Execute the generated code automatically
try:
    exec(generated_code, {})
    print("✅ Transformation complete. Check your output Excel file.")
except Exception as e:
    print("❌ Error running generated code:", e)


✅ Transformation complete. Check your output Excel file.
